In [5]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from gensim.utils import simple_preprocess
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
import numpy as np
import re
import underthesea # Thư viện tách từ
import pandas as pd
from sklearn.model_selection import train_test_split # Thư viện chia tách dữ liệu
from transformers import AutoModel, AutoTokenizer # Thư viện BERT
from underthesea import word_tokenize as word_tokenize_vn
# Thư viện train SVM
from sklearn.svm import SVC
from joblib import dump
from underthesea import sentiment
import re
import string
from underthesea import word_tokenize
import emoji  # Đảm bảo rằng thư viện này được cài đặt
from datetime import datetime, timedelta
import nltk
from symspellpy import SymSpell, Verbosity
import regex
import statistics 
import regex as re
from transformers import get_linear_schedule_with_warmup, AutoTokenizer, AutoModel, logging
from underthesea import word_tokenize
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from joblib import dump
from transformers import AutoModel, AutoTokenizer
import emoji  # Đảm bảo rằng thư viện này được cài đặt
from datetime import datetime, timedelta
import nltk
from symspellpy import SymSpell, Verbosity
import statistics 
import regex as re
from sklearn.model_selection import train_test_split, GridSearchCV
import warnings
warnings.filterwarnings("ignore")

logging.set_verbosity_error()

In [ ]:
def seed_everything(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed_everything(86)

Choose a maximum of length of 10 is safe.

In [10]:

file_path = 'Comment_segement.csv'
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

def load_bert():
    model = AutoModel.from_pretrained("vinai/phobert-base")
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return model, tokenizer

stopwords_file = r'D:/projects/NCKH/Data_Processing/vietnamese-stopwords.txt'
lookup_dict_file = r"D:\projects\NCKH\Data_Processing\lookup_dict.txt"

def load_lookup_dictionary(lookup_dict_file):
    lookup_dict = {}
    with open(lookup_dict_file, 'r', encoding='utf-8') as file:
        for line in file:
            if ':' in line:
                key, value = line.split(':', 1)  # Phân tách chỉ tại dấu hai chấm đầu tiên
                key = key.strip()  # Loại bỏ khoảng trắng thừa ở khoá
                value = value.strip()  # Loại bỏ khoảng trắng thừa ở giá trị
                lookup_dict[key] = value
    return lookup_dict
lookup_dict = load_lookup_dictionary(lookup_dict_file)

def load_stopwords(filepath="vietnamese-stopwords.txt"):
    with open(filepath, encoding='utf-8') as f:
        stopwords = [line.strip() for line in f.readlines()]
    return stopwords


def remove_stopwords(text, stopwords):
    # Tách từ và loại bỏ stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    # Nối các từ đã lọc thành một chuỗi mới
    filtered_text = ' '.join(filtered_words)
    return filtered_text
stopwords = load_stopwords(stopwords_file)

def remove_non_words(text):
    words = text.split()
    valid_words = [
        word for word in words 
        if 2 <= len(word) <= 8
        and not re.search(r'(.)\1{3,}', word)
        and len(set(word)) < 8 
    ]
    filtered_text = ' '.join(valid_words)
    return filtered_text

def split_sentences(text):
    # Xóa các ký tự xuống dòng
    cleaned_text = text.replace('\n', ' ')
    # Tách văn bản thành các câu bằng cách sử dụng dấu chấm
    sentences = cleaned_text.split('.')
    # Loại bỏ chuỗi trống và dư khoảng trắng
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

def remove_words(text):
    # Biểu thức chính quy để xác định từ bắt đầu bằng các ký tự có dấu
    pattern = r'\b[ÁáÀàẢảÃãẠạĂăẮắẰằẲẳẴẵẶặÂâẤấẦầẨẩẪẫẬậÉéÈèẺẻẼẽẸẹÊêẾếỀềỂểỄễỆệÍíÌìỈỉĨĩỊịÓóÒòỎỏÕõỌọÔôỐốỒồỔổỖỗỘộƠơỚớỜờỞởỠỡỢợÚúÙùỦủŨũỤụƯưỨứỪừỬửỮữỰựÝýỲỳỶỷỸỹỴỵ]\S*'
    # Loại bỏ các từ khớp với biểu thức chính quy ở đầu câu
    text = re.sub(pattern, '', text, count=1)
    return text

def normalize_text(text, lookup_dict):
    # Tách từng từ trong văn bản để kiểm tra từng từ có trong lookup_dict không
    words = text.split()
    # Thay thế từng từ nếu nó xuất hiện trong từ điển
    normalized_words = [lookup_dict[word] if word in lookup_dict else word for word in words]
    # Nối lại các từ đã chuẩn hóa thành một chuỗi mới
    normalized_text = ' '.join(normalized_words)
    return normalized_text

def replace_with_dictionary(text, dictionary):
    # Sắp xếp các cụm từ từ dài đến ngắn để đảm bảo rằng cụm từ dài nhất được áp dụng trước
    dictionary = sorted(dictionary, key=len, reverse=True)
    for phrase in dictionary:
        # Tạo mẫu regex để khớp cụm từ với khoảng trắng hoặc gạch dưới giữa các từ
        words = phrase.split()
        pattern = r'\b' + r'[_\s]*'.join([re.escape(word) for word in words]) + r'\b'
        replacement = "_".join(words)
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    return text

def standardize_data(row):
    row = row.lower()
    
    # Loại bỏ URLs and HTML
    row = re.sub(r'http\S+|www\S+|https\S+|<[^>]+>|\b(href|a)\b', '', row, flags=re.MULTILINE)
    # Loại bỏ ký tự đặc biệt
    row = re.sub(r'[^\w\s\.]', '', row, flags=re.UNICODE)
    # Loại bỏ từ lặp lại liên tiếp > 2
    row = re.sub(r'(.)\1{2,}', r'\1', row)
    # Thay thế chuỗi trắng liên tiếp bằng chuỗi trắng đơn
    row = re.sub(r'\s+', ' ', row)

    # Loại bỏ những từ xuất hiện < 6 kí tự và >2 kí tự
    row = remove_non_words(row)

    # Loại bỏ những từ có các nguyên âm đứng đầu câu
    row = remove_words(row)

    # Loại bỏ các ký tự tiếng Trung
    row = regex.sub(r'\p{Script=Han}+', '', row)
    # Loại bỏ các ký tự tiếng Nga (Cyrillic)
    row = regex.sub(r'\p{Script=Cyrillic}+', '', row)
    # Loại bỏ các ký tự tiếng Nhật
    row = regex.sub(r'\p{Script=Hiragana}+', '', row)
    row = regex.sub(r'\p{Script=Katakana}+', '', row)
    row = regex.sub(r'\p{IsHan}+', '', row)  # Bổ sung cho Kanji
    # Loại bỏ các ký tự tiếng Ấn Độ (Devanagari)
    row = regex.sub(r'\p{Script=Devanagari}+', '', row)
    
    # Loại bỏ các ký tự toán học in đậm
    row = regex.sub(r'\p{Math}', '', row)
    
    # Loại bỏ các ký tự không phải là chữ cái hoặc số
    row = regex.sub(r'[^\p{L}\p{N}\.]+', ' ', row)

    # Chuẩn hóa khoảng trắng
    row = re.sub(r'\s+', ' ', row).strip()

    # Convert emojis to row
    row = emoji.demojize(row, delimiters=("_", "_"))
    # Sử dụng regex để tìm và xóa các chuỗi emoji đã được chuẩn hóa
    row = re.sub(r'_[a-zA-Z0-9_]+_', '', row)

    # Xóa số
    row = re.sub("\d+", " ", row)

    row = normalize_text(row, lookup_dict)
    row = remove_stopwords(row, stopwords)
    row = re.sub(r'\bkhông\b\s*\?*\s*$', '', row)
    # Loại bỏ từ trùng lặp
    row = re.sub(r'\b(\w+)(?:\s+\1\b)+', r'\1', row)

    # Remove digits
    row = re.sub("\d+", " ", row)
    return row.strip().lower()

def load_data(filepath=file_path):
    df = pd.read_csv(filepath, encoding='utf-8')
    df['cleaned'] = df['sentence'].apply(standardize_data)
    return df['cleaned'].values, df['sentiment'].values

def make_bert_features(texts, tokenizer, model, max_len=20, stopwords=[]):
    tokenized = []
    for text in texts:
        # Loại bỏ stopwords
        text = " ".join([word for word in word_tokenize(text) if word.lower() not in stopwords])
        
        # Mã hóa câu văn thành token
        encoded = tokenizer.encode_plus(text, add_special_tokens=True, max_length=max_len, return_tensors="pt", pad_to_max_length=True, return_attention_mask=True, truncation=True)
        input_ids = encoded['input_ids']
        attention_mask = encoded['attention_mask']
        
        # Tính toán biểu diễn ngữ cảnh của câu
        with torch.no_grad():
            last_hidden_states = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # Lấy vectơ embedding của token [CLS]
        features = last_hidden_states[0][:, 0, :].detach().numpy()
        tokenized.append(features)
    
    # Xếp chồng các vectơ embedding của các câu
    return np.vstack(tokenized)

In [11]:
# Log messages
print("Chuẩn bị nạp danh sách các từ vô nghĩa (stopwords)...")
sw = load_stopwords()
print("Đã nạp xong danh sách các từ vô nghĩa")

print("Chuẩn bị nạp model BERT....")
phobert, tokenizer = load_bert()
print("Đã nạp xong model BERT.")

print("Chuẩn bị load dữ liệu....")
text, label = load_data()
print("Đã load dữ liệu xong")

print("Chuẩn bị tạo features từ BERT.....")
features = make_bert_features(text, tokenizer, phobert, stopwords=sw)
print("Đã tạo xong features từ BERT")

Chuẩn bị nạp danh sách các từ vô nghĩa (stopwords)...
Đã nạp xong danh sách các từ vô nghĩa
Chuẩn bị nạp model BERT....
Đã nạp xong model BERT.
Chuẩn bị load dữ liệu....
Đã load dữ liệu xong
Chuẩn bị tạo features từ BERT.....
Đã tạo xong features từ BERT


In [54]:
# Chia tập dữ liệu
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=86)

# Huấn luyện mô hình SVM
print("Chuẩn bị train model SVM....")
cl = SVC(kernel='linear', probability=True, gamma=0.125, C=1.0, class_weight='balanced')
cl.fit(features, label)

sc = cl.score(X_test, y_test)
print('Kết quả train model, độ chính xác = ', sc*100, '%')

# Save model
dump(cl, 'model_phoBERT.pkl')
print("Đã lưu model SVM vào file model_phoBERT.pkl.pkl")

Chuẩn bị train model SVM....
Kết quả train model, độ chính xác =  94.98607242339833 %
Đã lưu model SVM vào file model_phoBERT.pkl.pkl


In [12]:
from sklearn.metrics import accuracy_score, classification_report
from joblib import load
# Load model
loaded_model = load('model_phoBERT.pkl')
# Chia tập dữ liệu
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=86)

# Predict labels for the test dataset using the loaded model
y_pred = loaded_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of the model on the test dataset: {:.2f}%'.format(accuracy * 100))

# Calculate precision, recall, and F1-score
report = classification_report(y_test, y_pred, output_dict=False)  # Use output_dict=True to get the report as a dictionary
print('Classification Report:\n', report)


Accuracy of the model on the test dataset: 94.99%
Classification Report:
               precision    recall  f1-score   support

    negative       0.95      0.95      0.95      3037
    positive       0.95      0.95      0.95      3066

    accuracy                           0.95      6103
   macro avg       0.95      0.95      0.95      6103
weighted avg       0.95      0.95      0.95      6103



In [10]:
import os
import re
import torch
import numpy as np
import pandas as pd
from underthesea import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from joblib import dump
from transformers import AutoModel, AutoTokenizer
import regex
from joblib import load

# Load mô hình SVM
cl = load('model_phoBERT.pkl')

# Load PhoBERT và tokenizer
phobert, tokenizer = load_bert()  # Giả sử hàm load_bert() đã được định nghĩa như trước

# Hàm tiền xử lý câu văn mới
def preprocess_text(text, tokenizer, model, stopwords, max_len=20):
    text = standardize_data(text)  # Giả sử hàm này đã được định nghĩa
    text = " ".join([word for word in word_tokenize(text) if word.lower() not in stopwords])
    encoded = tokenizer.encode_plus(text, add_special_tokens=True, max_length=max_len, return_tensors="pt", padding='max_length', return_attention_mask=True, truncation=True)
    input_ids = encoded['input_ids']
    attention_mask = encoded['attention_mask']

    with torch.no_grad():
        last_hidden_states = model(input_ids=input_ids, attention_mask=attention_mask)
    features = last_hidden_states[0][:, 0, :].detach().numpy()
    return features

# Load danh sách stopwords
stopwords = load_stopwords()  # Giả sử hàm này đã được định nghĩa

# Tiền xử lý và dự đoán cho câu mới
new_sentence = "hàng không đẹp"
features_new = preprocess_text(new_sentence, tokenizer, phobert, stopwords)
prediction = cl.predict(features_new)

print("Dự đoán:", prediction)

Dự đoán: ['positive']


In [12]:
import os
import re
import torch
import numpy as np
import pandas as pd
from underthesea import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from joblib import dump, load
from transformers import AutoModel, AutoTokenizer

# Đọc dữ liệu
data = pd.read_csv(r"D:\projects\Cap1\phoBERT\phoBERT\TXL_Smarphone.csv")
data['content_cmt'] = data['content_cmt'].astype(str)

# Tải mô hình SVM và PhoBERT
cl = load('model_phoBERT.pkl')
phobert, tokenizer = load_bert()

# Hàm tiền xử lý và dự đoán
def preprocess_and_predict(text, tokenizer, model, stopwords, svm_model, max_len=20):
    if pd.isna(text):
        return None
    text = standardize_data(text)
    text = " ".join([word for word in word_tokenize(text) if word.lower() not in stopwords])
    encoded = tokenizer.encode_plus(
        text, add_special_tokens=True, max_length=max_len,
        return_tensors="pt", padding='max_length',
        return_attention_mask=True, truncation=True
    )
    input_ids = encoded['input_ids']
    attention_mask = encoded['attention_mask']
    with torch.no_grad():
        last_hidden_states = model(input_ids=input_ids, attention_mask=attention_mask)
    features = last_hidden_states[0][:, 0, :].detach().numpy()
    return svm_model.predict(features)[0]

# Tải danh sách stopwords
stopwords = load_stopwords()

# Dự đoán cảm xúc cho từng bình luận
data['sentiment_comment'] = data['content_cmt'].apply(lambda x: preprocess_and_predict(x, tokenizer, phobert, stopwords, cl))

# Lưu kết quả
data.to_csv(r"D:\projects\Cap1\phoBERT\phoBERT\TXL_Smarphone.csv", index=False, encoding='utf-8-sig')
